# 学習の準備から学習まで記載

## １．学習の準備

### クラスファイル作成

学習データのフォルダから作成。今回はフルーツの種類のみを対象に作成する。

#### モジュールインポート

In [ ]:
import os,sys
sys.path.append("../")

#### 定数宣言

In [ ]:
target_dir = '../../data/学習データ/479削除データ'
type_dir = os.path.join(target_dir, 'image')
class_file = 'type_class.txt'

In [ ]:
folders = os.listdir(type_dir)
last = folders[-1].split("_")[1]
with open(class_file, 'w', encoding='utf-8') as f:
    for folder in folders:
        target = folder.split("_")[1]
        f.write(target)
        print(target)
        if target != last:
            f.write('\n')

### 学習リスト作成
画像フォルダの構成から学習リストを作成する。
画像ファイルを検索し、その画像に該当するアノテーションファイルを探す。
"画像ファイルパス　アノテーション"のセットを1行としてファイルに書き出す。
以下、このファイルを学習ファイルと呼ぶ。

#### モジュールインポート
(後で切り出す用に上記と被るモジュールも記載)

In [ ]:
import os, glob

#### 定数宣言

In [ ]:
type_dir = '../train_set/学習データ/479削除データ'
train_file = 'train.txt'
class_file = 'type_class.txt'
img_dir = os.path.join(type_dir, 'image')

In [ ]:
class_list = []
with open(class_file, 'r', encoding='utf-8') as f:
    class_list = f.read().split("\n")

In [ ]:
print(class_list)

In [ ]:
train_list = []
error_list = []
for path in glob.glob(os.path.join(img_dir, '*')):
    
    if not os.path.isdir(path):
        continue
    class_name = path.split("/")[-1].split("_")[1]
    class_index = class_list.index(class_name)
    print(class_index, class_name) 
    for file in glob.glob(os.path.join(path, '**', '*.jpg'),recursive=True):
        annotation_file = file.replace("image", "annotation").replace(".jpg", ".txt")
        if not os.path.isfile(annotation_file):
            error_list.append(file)
            continue
            
        with open(annotation_file, 'r', encoding='UTF-8') as f:
            # Keras-yolo3のフォーマットに合わせる
            # x_min, y_min, x_max, y_maxの順
            line =  f.readline()
            while line:
                annotation = line.split(" ")
                x_min = annotation[0]
                x_max = annotation[1]
                y_min = annotation[2]
                y_max = annotation[3]
                train_list.append([file, x_min, y_min, x_max, y_max, str(class_index)])
                line = f.readline()

print("train_list:",len(train_list))
print("error_list:",len(error_list))
if error_list:
    print(error_list)

last = train_list[-1]
with open(train_file, 'w', encoding='utf-8') as f:
    for line in train_list:
        jpg = line[0]
        f.write(jpg+' ')
        f.write(','.join(line[1:]))
        if line != last:
            f.write('\n')
print("file made:",train_file)

## 学習

### モジュールインポート

In [ ]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import multi_gpu_model

from src.yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from src.yolo3.utils import get_random_data

### 関数追加
とりあえずデフォルト設定で何もいじらず使う

In [ ]:
def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)


def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    #model_body = multi_gpu_model(model_body, gpus=2)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)
    #model = multi_gpu_model(model, gpus=2)
    return model

def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

### 定数宣言

In [ ]:
annotation_path = 'train.txt' #学習リスト
log_dir = 'logs/マルチモデル/' #アウトプット先ふぉるだ
classes_path = 'type_class.txt' #クラスファイル
anchors_path = 'model_data/yolo_anchors.txt' #これはyolo設定なのでとりあえずデフォルト
first_weight = 'model_data/darknet53_weights.h5'

### ファイル等読み込み

In [ ]:
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

## 学習設定

In [ ]:
# 画像のサイズを416x416とする
input_shape = (416,416) # multiple of 32, hw

# モデルのインスタンス作成
model = create_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path=first_weight) # make sure you know what you freeze

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)

# ある条件で学習をストップさせる設定
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

# 訓練データと検証データに分けるとこ(とりあえずランダムで9:1)
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

## モデルコンパイルして学習

In [ ]:
# Train with frozen layers first, to get a stable loss.
# Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
if True:
    model.compile(optimizer=Adam(lr=1e-3), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: y_pred})

    batch_size = 8
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    history = model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=10,
            initial_epoch=0,
            callbacks=[logging, checkpoint])
    print(history)
    model.save_weights(log_dir + 'trained_weights_stage_1.h5')

# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
if True:
    #model = multi_gpu_model(model, gpus=2)
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')

    batch_size = 8 # note that more GPU memory is required after unfreezing the body
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=20,
        initial_epoch=10,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')

# Further training if needed.